In [1]:
#This is an engine for the game Othello
#To play the game, please make a copy of this project ("view" public access does not allow input())

In [2]:
!pip install aenum==3.1.5

You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


In [ ]:
#play the game here     
#make sure to run the block below first! 
game = Game(6)
game.play()

In [11]:
import enum 
from aenum import Enum, NoAlias
import random
import time
from copy import deepcopy

#enum Piece for representation
class Piece(Enum, settings = NoAlias):
    empty = " "
    white = "○"
    black = "●"
    potential_white = "⋅"
    potential_black = "⋅"

#class Game
class Game:

    board = None
    board_size = None
    move = "black"
    move_count = 0
    winner = None

    #init, BOARD_SIZE -> board dimension (default 8)
    def __init__(self, board_size = 8):
        if board_size % 2 != 0:
            print("board dimension must be an even number")
        elif board_size < 6 or board_size > 12:
            print("board dimension must be between 6 and 12")
        else: 
            self.board_size = board_size
            self.board = self.setup_board(board_size)

    #sets up the board, BOARD_SIZE -> board dimension
    def setup_board(self, board_size):
        empty_row = []
        for i in range(board_size):
            empty_row.append(Piece.empty)
        white_black_row = []
        for i in range(board_size):
            if i == (board_size / 2) - 1:
                white_black_row.append(Piece.white)
            elif i == (board_size / 2):
                white_black_row.append(Piece.black)
            else:
                white_black_row.append(Piece.empty)
        black_white_row = []
        for i in range(board_size):
            if i == (board_size / 2) - 1:
                black_white_row.append(Piece.black)
            elif i == (board_size / 2):
                black_white_row.append(Piece.white)
            else:
                black_white_row.append(Piece.empty)
        board = []
        for i in range(board_size):
            if i == (board_size / 2) - 1:
                board.append(white_black_row)
            elif i == board_size /2:
                board.append(black_white_row)
            else:
                board.append(empty_row.copy())
        return board

    #prints string representation of a row, N -> row index
    def print_row(self, n):
        top = "   +"
        bottom = str(n + 1) + "  │"
        for i in range(self.board_size):
            top += "-----+"
            bottom += "  " + self.board[n][i].value + "  │"
        print(top)
        print(bottom)
    
    #prints string representation of the entire board 
    def print_board(self):
        self.get_legal_positions(self.move)
        top = "    "
        bottom = "   +"
        for i in range(self.board_size):
            top += "  " + str(i + 1) + "   "
            bottom += "-----+"
        print(top)
        for i in range(self.board_size):
            self.print_row(i)
        print(bottom)
            
    #helper functino for getting potential moves, recursively finds legal positions based on DIRECTION to search
    def get_legal_positions_helper(self, direction, target_side, row0, col0):
        # 1 2 3
        # 4 x 5
        # 6 7 8
        if (self.board[row0][col0].name == "potential_black" or self.board[row0][col0].name == "potential_white"):
            return
        if direction == 1:
            if row0 - 1 >= 0 and col0 - 1 >= 0:
                if self.board[row0 - 1][col0 - 1].name == target_side:
                    self.get_legal_positions_helper(1, target_side, row0 - 1, col0 - 1)
                elif self.board[row0 - 1][col0 - 1].name == "empty":
                    if target_side == "white":
                        self.board[row0 - 1][col0 - 1] = Piece.potential_black
                    elif target_side == "black":
                        self.board[row0 - 1][col0 - 1] = Piece.potential_white
        elif direction == 2:
            if row0 - 1 >= 0:
                if self.board[row0 - 1][col0].name == target_side:
                    self.get_legal_positions_helper(2, target_side, row0 - 1, col0)
                elif self.board[row0 - 1][col0].name == "empty":
                    if target_side == "white":
                        self.board[row0 - 1][col0] = Piece.potential_black
                    elif target_side == "black":
                        self.board[row0 - 1][col0] = Piece.potential_white
        elif direction == 3:
            if row0 - 1 >= 0 and col0 + 1 < self.board_size:
                if self.board[row0 - 1][col0 + 1].name == target_side:
                    self.get_legal_positions_helper(3, target_side, row0 - 1, col0 + 1)
                elif self.board[row0 - 1][col0 + 1].name == "empty":
                    if target_side == "white":
                        self.board[row0 - 1][col0 + 1] = Piece.potential_black
                    elif target_side == "black":
                        self.board[row0 - 1][col0 + 1] = Piece.potential_white
        elif direction == 4:
            if col0 - 1 >= 0:
                if self.board[row0][col0 - 1].name == target_side:
                    self.get_legal_positions_helper(4, target_side, row0, col0 - 1)
                elif self.board[row0][col0 - 1].name == "empty":
                    if target_side == "white":
                        self.board[row0][col0 - 1] = Piece.potential_black
                    elif target_side == "black":
                        self.board[row0][col0 - 1] = Piece.potential_white
        elif direction == 5:
            if col0 + 1 < self.board_size:
                if self.board[row0][col0 + 1].name == target_side:
                    self.get_legal_positions_helper(5, target_side, row0, col0 + 1)
                elif self.board[row0][col0 + 1].name == "empty":
                    if target_side == "white":
                        self.board[row0][col0 + 1] = Piece.potential_black
                    elif target_side == "black":
                        self.board[row0][col0 + 1] = Piece.potential_white
        elif direction == 6:
            if row0 + 1 < self.board_size and col0 - 1 >= 0:
                if self.board[row0 + 1][col0 - 1].name == target_side:
                    self.get_legal_positions_helper(6, target_side, row0 + 1, col0 - 1)
                elif self.board[row0 + 1][col0 - 1].name == "empty":
                    if target_side == "white":
                        self.board[row0 + 1][col0 - 1] = Piece.potential_black
                    elif target_side == "black":
                        self.board[row0 + 1][col0 - 1] = Piece.potential_white
        elif direction == 7:
            if row0 + 1 < self.board_size:
                if self.board[row0 + 1][col0].name == target_side:
                    self.get_legal_positions_helper(7, target_side, row0 + 1, col0)
                elif self.board[row0 + 1][col0].name == "empty":
                    if target_side == "white":
                        self.board[row0 + 1][col0] = Piece.potential_black
                    elif target_side == "black":
                        self.board[row0 + 1][col0] = Piece.potential_white
        elif direction == 8:
            if row0 + 1 < self.board_size and col0 + 1 < self.board_size:
                if self.board[row0 + 1][col0 + 1].name == target_side:
                    self.get_legal_positions_helper(8, target_side, row0 + 1, col0 + 1)
                elif self.board[row0 + 1][col0 + 1].name == "empty":
                    if target_side == "white":
                        self.board[row0 + 1][col0 + 1] = Piece.potential_black
                    elif target_side == "black":
                        self.board[row0 + 1][col0 + 1] = Piece.potential_white
        else:
            print("Error: direction does not exist")
        
    #finds all legal positions for SIDE and marks them on the board
    def get_legal_positions(self, side): 
        if side == "black":
            target_side = "white"
        elif side == "white":
            target_side = "black"
        else:
            print("Error: side is neither black or white")
            return
        for i in range(self.board_size):
                for j in range(self.board_size):
                    if self.board[i][j].name == side:
                        if (i - 1 >= 0 and j - 1 >= 0) and self.board[i - 1][j - 1].name == target_side:
                            self.get_legal_positions_helper(1, target_side, i - 1, j - 1)
                        if (i - 1 >= 0) and self.board[i - 1][j].name == target_side:
                            self.get_legal_positions_helper(2, target_side, i - 1, j)
                        if (i - 1 >= 0 and j + 1 < self.board_size) and self.board[i - 1][j + 1].name == target_side:
                            self.get_legal_positions_helper(3, target_side, i - 1, j + 1)
                        if (j - 1 >= 0) and self.board[i][j - 1].name == target_side:
                            self.get_legal_positions_helper(4, target_side, i, j - 1)
                        if (j + 1 < self.board_size) and self.board[i][j + 1].name == target_side:
                            self.get_legal_positions_helper(5, target_side, i, j + 1)
                        if (i + 1 < self.board_size and j - 1 >= 0) and self.board[i + 1][j - 1].name == target_side:
                            self.get_legal_positions_helper(6, target_side, i + 1, j - 1)
                        if (i + 1 < self.board_size) and self.board[i + 1][j].name == target_side:
                            self.get_legal_positions_helper(7, target_side, i + 1, j)
                        if (i + 1 < self.board_size and j + 1 < self.board_size) and self.board[i + 1][j + 1].name == target_side:
                            self.get_legal_positions_helper(8, target_side, i + 1, j + 1)

    def flip(self, side, row0, col0):
        if side == "black":
            target_side = "white"
        elif side == "white":
            target_side = "black"
        #1
        if (row0 - 1 >= 0 and col0 - 1 >= 0) and self.board[row0 - 1][col0 - 1].name == target_side:
            i, j, distance = row0 - 1, col0 - 1, 0
            while (i >= 0 and j >= 0):
                if self.board[i][j].name == side:
                    break
                elif self.board[i][j].name == target_side:
                    i -= 1
                    j -= 1
                    distance += 1
                elif self.board[i][j].name != side and self.board[i][j].name != target_side:
                    distance = 0
                    break
            if i < 0 or j < 0:
                distance = 0
            i, j = row0 - 1, col0 - 1
            for k in range(distance):
                self.board[i][j] = Piece[side]
                i -= 1
                j -= 1
        #2
        if (row0 - 1 >= 0) and self.board[row0 - 1][col0].name == target_side:
            i, j, distance = row0 - 1, col0, 0
            while (i >= 0):
                if self.board[i][j].name == side:
                    break
                elif self.board[i][j].name == target_side:
                    i -= 1
                    distance += 1
                elif self.board[i][j].name != side and self.board[i][j].name != target_side:
                    distance = 0
                    break
            if i < 0:
                distance = 0
            i, j = row0 - 1, col0
            for k in range(distance):
                self.board[i][j] = Piece[side]
                i -= 1
        #3
        if (row0 - 1 >= 0 and col0 + 1 < self.board_size) and self.board[row0 - 1][col0 + 1].name == target_side:
            i, j, distance = row0 - 1, col0 + 1, 0
            while (i >= 0 and j < self.board_size):
                if self.board[i][j].name == side:
                    break
                elif self.board[i][j].name == target_side:
                    i -= 1
                    j += 1
                    distance += 1
                elif self.board[i][j].name != side and self.board[i][j].name != target_side:
                    distance = 0
                    break
            if i < 0 or j == self.board_size:
                distance = 0
            i, j = row0 - 1, col0 + 1
            for k in range(distance):
                self.board[i][j] = Piece[side]
                i -= 1
                j += 1
        #4
        if (col0 - 1 >= 0) and self.board[row0][col0 - 1].name == target_side:
            i, j, distance = row0, col0 - 1, 0
            while (j >= 0):
                if self.board[i][j].name == side:
                    break
                elif self.board[i][j].name == target_side:
                    j -= 1
                    distance += 1
                elif self.board[i][j].name != side and self.board[i][j].name != target_side:
                    distance = 0
                    break
            if j < 0:
                distance = 0
            i, j = row0, col0 - 1
            for k in range(distance):
                self.board[i][j] = Piece[side]
                j -= 1
        #5
        if (col0 + 1 < self.board_size) and self.board[row0][col0 + 1].name == target_side:
            i, j, distance = row0, col0 + 1, 0
            while (j < self.board_size):
                if self.board[i][j].name == side:
                    break
                elif self.board[i][j].name == target_side:
                    j += 1
                    distance += 1
                elif self.board[i][j].name != side and self.board[i][j].name != target_side:
                    distance = 0
                    break
            if j == self.board_size:
                distance = 0
            i, j = row0, col0 + 1
            for k in range(distance):
                self.board[i][j] = Piece[side]
                j += 1
        #6
        if (row0 + 1 < self.board_size and col0 - 1 >= 0) and self.board[row0 + 1][col0 - 1].name == target_side:
            i, j, distance = row0 + 1, col0 - 1, 0
            while (i < self.board_size and j >= 0):
                if self.board[i][j].name == side:
                    break
                elif self.board[i][j].name == target_side:
                    i += 1
                    j -= 1
                    distance += 1
                elif self.board[i][j].name != side and self.board[i][j].name != target_side:
                    distance = 0
                    break
            if i == self.board_size or j < 0:
                distance = 0
            i, j = row0 + 1, col0 - 1
            for k in range(distance):
                self.board[i][j] = Piece[side]
                i += 1
                j -= 1
        #7
        if (row0 + 1 < self.board_size) and self.board[row0 + 1][col0].name == target_side:
            i, j, distance = row0 + 1, col0, 0
            while (i < self.board_size):
                if self.board[i][j].name == side:
                    break
                elif self.board[i][j].name == target_side:
                    i += 1
                    distance += 1
                elif self.board[i][j].name != side and self.board[i][j].name != target_side:
                    distance = 0
                    break
            if i == self.board_size:
                distance = 0
            i, j = row0 + 1, col0
            for k in range(distance):
                self.board[i][j] = Piece[side]
                i += 1
        #8
        if (row0 + 1 < self.board_size and col0 + 1 < self.board_size) and self.board[row0 + 1][col0 + 1].name == target_side:
            i, j, distance = row0 + 1, col0 + 1, 0
            while (i < self.board_size and j < self.board_size):
                if self.board[i][j].name == side:
                    break
                elif self.board[i][j].name == target_side:
                    i += 1
                    j += 1
                    distance += 1
                elif self.board[i][j].name != side and self.board[i][j].name != target_side:
                    distance = 0
                    break
            if i == self.board_size or j == self.board_size:
                distance = 0
            i, j = row0 + 1, col0 + 1
            for k in range(distance):
                self.board[i][j] = Piece[side]
                i += 1
                j += 1

    def clean_potentials(self):
        for i in range(self.board_size):
            for j in range(self.board_size):
                if self.board[i][j].name == "potential_black" or self.board[i][j].name == "potential_white":
                    self.board[i][j] = Piece.empty

    def count_pieces(self):
        black_count, white_count = 0, 0
        for i in range(self.board_size):
            for j in range(self.board_size):
                if self.board[i][j].name == "black": 
                    black_count += 1
                elif self.board[i][j].name == "white": 
                    white_count += 1
        result_dict = {
            "message": ("Score: Black " + str(black_count) + " - " + str(white_count) + " White"),
            "black_count": black_count,
            "white_count": white_count,
            "total": black_count + white_count
        }
        return result_dict

    def exists_available_moves(self, side):
        for i in range(self.board_size):
            for j in range(self.board_size):
                if side == "black" and self.board[i][j].name == "potential_black":
                    return True
                elif side == "white" and self.board[i][j].name == "potential_white":
                    return True
        return False

    #makes a move by adding a piece to [ROW, COL] depending on whose turn it is, 
    #changes aboard accordingly anc checks legality
    def make_move(self, row, col):
        if self.move == "white":
            print("--------------------------------------------------------------------------")
            if self.board[row - 1][col - 1].name == "potential_white":
                self.board[row - 1][col - 1] = Piece.white
                self.flip("white", row - 1, col - 1)
                self.clean_potentials()
                self.move = "black"
                self.move_count += 1
                print("Move " + str(self.move_count) + 
                    ": White moved [" + str(row) + ", " + str(col) + "]")
                result_dict = self.count_pieces()
                print(result_dict["message"])
                self.print_board()
                if result_dict["total"] == self.board_size * self.board_size:
                    if result_dict["black_count"] > result_dict["white_count"]:
                        print("Game Over, Black won!")
                        self.winner = "black"
                    elif result_dict["black_count"] < result_dict["white_count"]:
                        print("Game Over, White won!")
                        self.winner = "white"
                    else:
                        print("Game Over, it's a tie!")
                        self.winner = "tie"
                elif self.exists_available_moves("black") == False:
                    self.move = "white"
                    print()
                    print("No moves available for Black, White's turn to move")
                    self.print_board()
                else:
                    print("Black's turn to move")
                print()
                return True
            else:
                print("Error: illegal move (square is not available for white)")
                return False
        elif self.move == "black": 
            print("--------------------------------------------------------------------------")
            if self.board[row - 1][col - 1].name == "potential_black":
                self.board[row - 1][col - 1] = Piece.black
                self.flip("black", row - 1, col - 1)
                self.clean_potentials()
                self.move = "white"
                self.move_count += 1
                print("Move " + str(self.move_count) + 
                    ": Black moved [" + str(row) + ", " + str(col) + "]")
                result_dict = self.count_pieces()
                print(result_dict["message"])
                self.print_board()
                if result_dict["total"] == self.board_size * self.board_size:
                    if result_dict["black_count"] > result_dict["white_count"]:
                        print("Game Over, Black won!")
                        self.winner = "black"
                    elif result_dict["black_count"] < result_dict["white_count"]:
                        print("Game Over, White won!")
                        self.winner = "white"
                    else:
                        print("Game Over, it's a tie!")
                        self.winner = "tie"
                elif self.exists_available_moves("white") == False:
                    self.move = "black"
                    print()
                    print("No moves available for White, Black's turn to move")
                    self.print_board()
                else:
                    print("White's turn to move")
                print()
                return True
            else:
                print("Error: illegal move (square is not available for black)")
                return False
        else:
            print("Error: neither's move")
            return False
        
    #prints inital state of game
    def preface(self): 
        print("Black pieces are marked with ●, White pieces are marked with ○")
        print("Black's turn to move, available moves are marked with ⋅")
        self.print_board()
        print()

    #gets row and col from input
    def get_row_and_col(self, move):
        move = "[" + move + "]"
        if move[2] == ",":
            #[1, 2] and [1, 20]
            row = move[1: 2]
            col = move[4: 6]
            if col[1] == "]":
                col = col[0]
            row = int(row)
            col = int(col)
            return (row, col)
        elif move[3] == ',':
            #[10, 2] and [10, 20]
            row = move[1: 3]
            col = move[5: 7]
            if col[1] == "]":
                col = col[0]
            row = int(row)
            col = int(col)
            return (row, col)
        else:
            print("Error: wrong input format")
            return (0, 0)

    def reset(self):
        self.board = None
        self.move = "black"
        self.move_count = 0
        self.winner = None
        self.board = self.setup_board(self.board_size)

    def play_two_players(self):
        self.preface()
        while self.winner == None:
            move = input("Enter a move: [x, y]")
            if len(move) < 4 or len(move) > 6:
                print("--------------------------------------------------------------------------")
                print("Error: bad input")
                continue
            rc_tuple = self.get_row_and_col(move)
            if rc_tuple == (0, 0): 
                print("--------------------------------------------------------------------------")
                print("Error: bad input")
                continue
            row = rc_tuple[0]
            col = rc_tuple[1]
            self.make_move(row, col)
        restart = input("Restart? [y/n]")
        while restart != "y" and restart != "n":
            print("--------------------------------------------------------------------------")
            print("Error: bad input")
            restart = input("Restart? [y/n]")
        if restart == "y":
            self.reset()
            self.play()

   

    def get_random_move(self):
        result = []
        for i in range(self.board_size):
            for j in range(self.board_size):
                if self.board[i][j].name == "potential_black" and self.move == "black":
                    tup = (i, j)
                    result.append(tup)
                elif self.board[i][j].name == "potential_white" and self.move == "white":
                    tup = (i, j)
                    result.append(tup)
        return random.choice(result)

    def test_run_random(self):
        self.preface()
        while self.winner == None:
            print(self.move)
            move = self.get_random_move()
            row = move[0] + 1
            col = move[1] + 1
            print(move)
            self.make_move(row, col)
    
    def copy_board(self, old_board):
        new_board = [row[:] for row in old_board]
        return new_board

    best_move = None

    def ai_add_piece(self, old_board, side, row0, col0):
        new_board = self.copy_board(old_board)
        if side == "black":
            new_board[row0][col0] = Piece.black
            self.ai_flip(new_board, "black", row0, col0)
        elif side == "white":
            new_board[row0][col0] = Piece.white
            self.ai_flip(new_board, "white", row0, col0)
        return new_board

    def ai_count_pieces(self, board):
        black_count, white_count = 0, 0
        for i in range(self.board_size):
            for j in range(self.board_size):
                if board[i][j].name == "black": 
                    black_count += 1
                elif board[i][j].name == "white": 
                    white_count += 1
        result_dict = {
            "message": ("Score: Black " + str(black_count) + " - " + str(white_count) + " White"),
            "black_count": black_count,
            "white_count": white_count,
            "total": black_count + white_count
        }
        return result_dict

    #helper functino for getting potential moves, recursively finds legal positions based on DIRECTION to search
    def ai_get_legal_positions_helper(self, direction, board, target_side, row0, col0, array):
        # 1 2 3
        # 4 x 5
        # 6 7 8
        if direction == 1:
            if row0 - 1 >= 0 and col0 - 1 >= 0:
                if board[row0 - 1][col0 - 1].name == target_side:
                    self.ai_get_legal_positions_helper(1, board, target_side, row0 - 1, col0 - 1, array)
                elif board[row0 - 1][col0 - 1].name == "empty" and (row0 - 1, col0 - 1) not in array:
                    array.append((row0 - 1, col0 - 1))
        elif direction == 2:
            if row0 - 1 >= 0:
                if board[row0 - 1][col0].name == target_side:
                    self.ai_get_legal_positions_helper(2, board, target_side, row0 - 1, col0, array)
                elif board[row0 - 1][col0].name == "empty" and (row0 - 1, col0 ) not in array:
                    array.append((row0 - 1, col0))
        elif direction == 3:
            if row0 - 1 >= 0 and col0 + 1 < self.board_size:
                if board[row0 - 1][col0 + 1].name == target_side:
                    self.ai_get_legal_positions_helper(3, board, target_side, row0 - 1, col0 + 1, array)
                elif board[row0 - 1][col0 + 1].name == "empty" and (row0 - 1, col0 + 1) not in array:
                    array.append((row0 - 1, col0 + 1))
        elif direction == 4:
            if col0 - 1 >= 0:
                if board[row0][col0 - 1].name == target_side:
                    self.ai_get_legal_positions_helper(4, board, target_side, row0, col0 - 1, array)
                elif board[row0][col0 - 1].name == "empty" and (row0, col0 - 1) not in array:
                    array.append((row0, col0 - 1))
        elif direction == 5:
            if col0 + 1 < self.board_size:
                if board[row0][col0 + 1].name == target_side:
                    self.ai_get_legal_positions_helper(5, board, target_side, row0, col0 + 1, array)
                elif board[row0][col0 + 1].name == "empty" and (row0, col0 + 1) not in array:
                    array.append((row0, col0 + 1))
        elif direction == 6:
            if row0 + 1 < self.board_size and col0 - 1 >= 0:
                if board[row0 + 1][col0 - 1].name == target_side:
                    self.ai_get_legal_positions_helper(6, board, target_side, row0 + 1, col0 - 1, array)
                elif board[row0 + 1][col0 - 1].name == "empty" and (row0 + 1, col0 - 1) not in array:
                    array.append((row0 + 1, col0 - 1))
        elif direction == 7:
            if row0 + 1 < self.board_size:
                if board[row0 + 1][col0].name == target_side:
                    self.ai_get_legal_positions_helper(7, board, target_side, row0 + 1, col0, array)
                elif board[row0 + 1][col0].name == "empty" and (row0 + 1, col0) not in array:
                    array.append((row0 + 1, col0))
        elif direction == 8:
            if row0 + 1 < self.board_size and col0 + 1 < self.board_size:
                if board[row0 + 1][col0 + 1].name == target_side:
                    self.ai_get_legal_positions_helper(8, board, target_side, row0 + 1, col0 + 1, array)
                elif board[row0 + 1][col0 + 1].name == "empty" and (row0 + 1, col0 + 1) not in array:
                    array.append((row0 + 1, col0 + 1))
        else:
            print("Error: direction does not exist")
        
    #finds all legal positions for SIDE and returns array of all positions
    def ai_get_legal_positions(self, board, side): 
        if side == "black":
            target_side = "white"
        elif side == "white":
            target_side = "black"
        else:
            print("Error: side is neither black or white")
            return
        array = []
        for i in range(self.board_size):
                for j in range(self.board_size):
                    if board[i][j].name == side:
                        if (i - 1 >= 0 and j - 1 >= 0) and board[i - 1][j - 1].name == target_side:
                            self.ai_get_legal_positions_helper(1, board, target_side, i - 1, j - 1, array)
                        if (i - 1 >= 0) and board[i - 1][j].name == target_side:
                            self.ai_get_legal_positions_helper(2, board, target_side, i - 1, j, array)
                        if (i - 1 >= 0 and j + 1 < self.board_size) and board[i - 1][j + 1].name == target_side:
                            self.ai_get_legal_positions_helper(3, board, target_side, i - 1, j + 1, array)
                        if (j - 1 >= 0) and board[i][j - 1].name == target_side:
                            self.ai_get_legal_positions_helper(4, board, target_side, i, j - 1, array)
                        if (j + 1 < self.board_size) and board[i][j + 1].name == target_side:
                            self.ai_get_legal_positions_helper(5, board, target_side, i, j + 1, array)
                        if (i + 1 < self.board_size and j - 1 >= 0) and board[i + 1][j - 1].name == target_side:
                            self.ai_get_legal_positions_helper(6, board, target_side, i + 1, j - 1, array)
                        if (i + 1 < self.board_size) and board[i + 1][j].name == target_side:
                            self.ai_get_legal_positions_helper(7, board, target_side, i + 1, j, array)
                        if (i + 1 < self.board_size and j + 1 < self.board_size) and board[i + 1][j + 1].name == target_side:
                            self.ai_get_legal_positions_helper(8, board, target_side, i + 1, j + 1, array)
        return array
    
    #prints string representation of a row, N -> row index
    def ai_print_row(self, board, n):
        top = "   +"
        bottom = str(n + 1) + "  │"
        for i in range(self.board_size):
            top += "-----+"
            bottom += "  " + board[n][i].value + "  │"
        print(top)
        print(bottom)
    
    #prints string representation of the entire board 
    def ai_print_board(self, board):
        top = "    "
        bottom = "   +"
        for i in range(self.board_size):
            top += "  " + str(i + 1) + "   "
            bottom += "-----+"
        print(top)
        for i in range(self.board_size):
            self.ai_print_row(board, i)
        print(bottom)

    def ai_flip(self, board, side, row0, col0):
        if side == "black":
            target_side = "white"
        elif side == "white":
            target_side = "black"
        #1
        if (row0 - 1 >= 0 and col0 - 1 >= 0) and board[row0 - 1][col0 - 1].name == target_side:
            i, j, distance = row0 - 1, col0 - 1, 0
            while (i >= 0 and j >= 0):
                if board[i][j].name == side:
                    break
                elif board[i][j].name == target_side:
                    i -= 1
                    j -= 1
                    distance += 1
                elif board[i][j].name != side and board[i][j].name != target_side:
                    distance = 0
                    break
            if i < 0 or j < 0:
                distance = 0
            i, j = row0 - 1, col0 - 1
            for k in range(distance):
                board[i][j] = Piece[side]
                i -= 1
                j -= 1
        #2
        if (row0 - 1 >= 0) and board[row0 - 1][col0].name == target_side:
            i, j, distance = row0 - 1, col0, 0
            while (i >= 0):
                if board[i][j].name == side:
                    break
                elif board[i][j].name == target_side:
                    i -= 1
                    distance += 1
                elif board[i][j].name != side and board[i][j].name != target_side:
                    distance = 0
                    break
            if i < 0:
                distance = 0
            i, j = row0 - 1, col0
            for k in range(distance):
                board[i][j] = Piece[side]
                i -= 1
        #3
        if (row0 - 1 >= 0 and col0 + 1 < self.board_size) and board[row0 - 1][col0 + 1].name == target_side:
            i, j, distance = row0 - 1, col0 + 1, 0
            while (i >= 0 and j < self.board_size):
                if board[i][j].name == side:
                    break
                elif board[i][j].name == target_side:
                    i -= 1
                    j += 1
                    distance += 1
                elif board[i][j].name != side and board[i][j].name != target_side:
                    distance = 0
                    break
            if i < 0 or j == self.board_size:
                distance = 0
            i, j = row0 - 1, col0 + 1
            for k in range(distance):
                board[i][j] = Piece[side]
                i -= 1
                j += 1
        #4
        if (col0 - 1 >= 0) and board[row0][col0 - 1].name == target_side:
            i, j, distance = row0, col0 - 1, 0
            while (j >= 0):
                if board[i][j].name == side:
                    break
                elif board[i][j].name == target_side:
                    j -= 1
                    distance += 1
                elif board[i][j].name != side and board[i][j].name != target_side:
                    distance = 0
                    break
            if j < 0:
                distance = 0
            i, j = row0, col0 - 1
            for k in range(distance):
                board[i][j] = Piece[side]
                j -= 1
        #5
        if (col0 + 1 < self.board_size) and board[row0][col0 + 1].name == target_side:
            i, j, distance = row0, col0 + 1, 0
            while (j < self.board_size):
                if board[i][j].name == side:
                    break
                elif board[i][j].name == target_side:
                    j += 1
                    distance += 1
                elif board[i][j].name != side and self.board[i][j].name != target_side:
                    distance = 0
                    break
            if j == self.board_size:
                distance = 0
            i, j = row0, col0 + 1
            for k in range(distance):
                board[i][j] = Piece[side]
                j += 1
        #6
        if (row0 + 1 < self.board_size and col0 - 1 >= 0) and board[row0 + 1][col0 - 1].name == target_side:
            i, j, distance = row0 + 1, col0 - 1, 0
            while (i < self.board_size and j >= 0):
                if board[i][j].name == side:
                    break
                elif board[i][j].name == target_side:
                    i += 1
                    j -= 1
                    distance += 1
                elif board[i][j].name != side and board[i][j].name != target_side:
                    distance = 0
                    break
            if i == self.board_size or j < 0:
                distance = 0
            i, j = row0 + 1, col0 - 1
            for k in range(distance):
                board[i][j] = Piece[side]
                i += 1
                j -= 1
        #7
        if (row0 + 1 < self.board_size) and board[row0 + 1][col0].name == target_side:
            i, j, distance = row0 + 1, col0, 0
            while (i < self.board_size):
                if board[i][j].name == side:
                    break
                elif board[i][j].name == target_side:
                    i += 1
                    distance += 1
                elif board[i][j].name != side and board[i][j].name != target_side:
                    distance = 0
                    break
            if i == self.board_size:
                distance = 0
            i, j = row0 + 1, col0
            for k in range(distance):
                board[i][j] = Piece[side]
                i += 1
        #8
        if (row0 + 1 < self.board_size and col0 + 1 < self.board_size) and board[row0 + 1][col0 + 1].name == target_side:
            i, j, distance = row0 + 1, col0 + 1, 0
            while (i < self.board_size and j < self.board_size):
                if board[i][j].name == side:
                    break
                elif board[i][j].name == target_side:
                    i += 1
                    j += 1
                    distance += 1
                elif board[i][j].name != side and board[i][j].name != target_side:
                    distance = 0
                    break
            if i == self.board_size or j == self.board_size:
                distance = 0
            i, j = row0 + 1, col0 + 1
            for k in range(distance):
                board[i][j] = Piece[side]
                i += 1
                j += 1

    #sets best move
    def ai_set_move(self, board):
        if self.move == "black":
            val = self.ai_min_max(board, 4, True, 1, -10000, 10000)
        elif self.move == "white":
            val = self.ai_min_max(board, 4, True, -1, -10000, 10000)
        else:
            print("Error: cannot find a move case 1")
        if self.best_move == None:
            print("Error: cannot find a move case 2")
        return val;
    
    def copy_clean_board(self):
        clean_board = self.copy_board(self.board)
        for i in range(self.board_size):
            for j in range(self.board_size):
                if clean_board[i][j].name == "potential_black" or clean_board[i][j].name == "potential_white":
                    clean_board[i][j] = Piece.empty
        return clean_board
    
    def ai_make_move(self):
        if self.winner == None:
            board = self.copy_clean_board()
            val = self.ai_set_move(board)
            #print(self.best_move[0] + 1, self.best_move[1] + 1)
            #print(self.move, val)
            self.make_move(self.best_move[0] + 1, self.best_move[1] + 1)
            self.best_move = None

    def ai_get_winner(self, count_dict):
        if count_dict["total"] == self.board_size * self.board_size:
            if black_count > white_count:
                return "black"
            elif black_count < white_count:
                return "white"
        else:
            return None

    def ai_min_max(self, board, depth, save_move, sense, alpha, beta):
        count_dict = self.ai_count_pieces(board)
        if depth == 0 or (count_dict["total"] == self.board_size * self.board_size):
            return self.ai_heuristic(board, count_dict)
        if sense == 1:
            best_so_far = -10000
            possible_moves = self.ai_get_legal_positions(board, "black")
            for move in possible_moves:
                child_board = self.copy_board(board)
                child_board = self.ai_add_piece(child_board, "black", move[0], move[1])
                if save_move and self.ai_get_winner(count_dict) == "black":
                    self.best_move = move
                    break
                # if save_move:
                #     self.ai_print_board(child_board)
                response = self.ai_min_max(child_board, depth - 1, False, -1, alpha, beta)
                if save_move and response >= best_so_far:
                    self.best_move = move
                best_so_far = max(response, best_so_far)
                alpha = max(best_so_far, alpha)
                if alpha > beta:
                    break
            if len(possible_moves) == 0:
                response = self.ai_min_max(board, depth - 1, False, -1, alpha, beta)
                best_so_far = max(response, best_so_far)
            return best_so_far
        elif sense == -1:
            best_so_far = 10000
            possible_moves = self.ai_get_legal_positions(board, "white")
            for move in possible_moves:
                child_board = self.copy_board(board)
                child_board = self.ai_add_piece(child_board, "white", move[0], move[1])
                if save_move and self.ai_get_winner(count_dict) == "white":
                    self.best_move = move
                    break
                # if save_move:
                #     self.ai_print_board(child_board)
                response = self.ai_min_max(child_board, depth - 1, False, 1, alpha, beta)
                if save_move and response <= best_so_far:
                    self.best_move = move
                best_so_far = min(response, best_so_far)
                beta = min(best_so_far, beta)
                if alpha > beta:
                    break
            if len(possible_moves) == 0:
                response = self.ai_min_max(board, depth - 1, False, 1, alpha, beta)
                best_so_far = min(response, best_so_far)
            return best_so_far
        else:
            print("Error: sense is neither 1 nor -1")

     #------------------------------------------------------
    #main program to execute
    def play(self):
        game_mode = input("Select game mode: [1p/2p]")
        while game_mode != "1p" and game_mode != "2p":
            print("--------------------------------------------------------------------------")
            print("Error: bad input")
            game_mode = input("Select game mode: [1p/2p]")
        if game_mode == "1p":
            self.play_ai()
        elif game_mode == "2p":
            self.play_two_players()
    #------------------------------------------------------

    def play_ai(self):
        player = random.choice(["White", "Black"])
        if player == "Black":
            computer = "White"
        else:
            computer = "Black"
        if player == "Black":
            self.preface()
            print("Player is " + player + ", Computer is " + computer)
            while self.winner == None:
                if player == "Black" and self.move == "black" and self.winner == None:
                    moved = False
                    while moved == False:
                        move = input("Enter a move: [x, y]")
                        if len(move) < 4 or len(move) > 6:
                            print("--------------------------------------------------------------------------")
                            print("Error: bad input")
                            while len(move) < 4 or len(move) > 6:
                                move = input("Enter a move: [x, y]")
                        rc_tuple = self.get_row_and_col(move)
                        if rc_tuple == (0, 0): 
                            print("--------------------------------------------------------------------------")
                            print("Error: bad input")
                            while rc_tuple == (0, 0):
                                rc_tuple = self.get_row_and_col(move)
                        row = rc_tuple[0]
                        col = rc_tuple[1]
                        moved = self.make_move(row, col)
                if computer == "White" and self.move == "white" and self.winner == None:
                    self.ai_make_move()
        elif player == "White":
            self.preface()
            print("Player is " + player + ", Computer is " + computer)
            while self.winner == None:
                if computer == "Black" and self.move == "black" and self.winner == None:
                    self.ai_make_move()
                if player == "White" and self.move == "white" and self.winner == None:
                    moved = False
                    while moved == False:
                        move = input("Enter a move: [x, y]")
                        if len(move) < 4 or len(move) > 6:
                            print("--------------------------------------------------------------------------")
                            print("Error: bad input")
                            while len(move) < 4 or len(move) > 6:
                                move = input("Enter a move: [x, y]")
                        rc_tuple = self.get_row_and_col(move)
                        if rc_tuple == (0, 0): 
                            print("--------------------------------------------------------------------------")
                            print("Error: bad input")
                            while rc_tuple == (0, 0):
                                rc_tuple = self.get_row_and_col(move)
                        row = rc_tuple[0]
                        col = rc_tuple[1]
                        moved = self.make_move(row, col)
        else:
            print("some werid error")
            return
        restart = input("Restart? [y/n]")
        while restart != "y" and restart != "n":
            print("--------------------------------------------------------------------------")
            print("Error: bad input")
            restart = input("Restart? [y/n]")
        if restart == "y":
            self.reset()
            self.play()
    
    def get_corner_score(self, board):
        black_corner = 0
        white_corner = 0
        if board[0][0] == Piece.black:
            black_corner += 1
        elif board[0][0] == Piece.white:
            white_corner += 1
        if board[0][self.board_size - 1] == Piece.black:
            black_corner += 1
        elif board[0][self.board_size - 1] == Piece.white:
            white_corner += 1
        if board[self.board_size - 1][0] == Piece.black:
            black_corner += 1
        elif board[self.board_size - 1][0] == Piece.white:
            white_corner += 1
        if board[self.board_size - 1][self.board_size - 1] == Piece.black:
            black_corner += 1
        elif board[self.board_size - 1][self.board_size - 1] == Piece.white:
            white_corner += 1
        return (black_corner - white_corner) / 4 * 100

    def get_x_squares_score(self, board):
        black_x = 0
        white_x = 0
        if board[1][0] == Piece.black:
            black_x += 1
        elif board[1][0] == Piece.white:
            white_x += 1
        if board[0][1] == Piece.black:
            black_x += 1
        elif board[0][1] == Piece.white:
            white_x += 1
        if board[1][self.board_size - 1] == Piece.black:
            black_x += 1
        elif board[1][self.board_size - 1] == Piece.white:
            white_x += 1
        if board[0][self.board_size - 2] == Piece.black:
            black_x += 1
        elif board[0][self.board_size - 2] == Piece.white:
            white_x += 1
        if board[self.board_size - 2][0] == Piece.black:
            black_x += 1
        elif board[self.board_size - 2][0] == Piece.white:
            white_x += 1
        if board[self.board_size - 1][1] == Piece.black:
            black_x += 1
        elif board[self.board_size - 1][1] == Piece.white:
            white_x += 1
        if board[self.board_size - 2][self.board_size - 1] == Piece.black:
            black_x += 1
        elif board[self.board_size - 2][self.board_size - 1] == Piece.white:
            white_x += 1
        if board[self.board_size - 1][self.board_size -  2] == Piece.black:
            black_x += 1
        elif board[self.board_size - 1][self.board_size - 2] == Piece.white:
            white_x += 1
        return (white_x - black_x) / 8 * 100

    def get_edge_score(self, board):
        black_edge = 0
        white_edge = 0
        row0 = 0
        for col0 in range(self.board_size):
            if board[row0][col0] == Piece.black:
                black_edge += 1
            elif board[row0][col0] == Piece.white:
                white_edge += 1
        row0 = self.board_size - 1
        for col0 in range(self.board_size):
            if board[row0][col0] == Piece.black:
                black_edge += 1
            elif board[row0][col0] == Piece.white:
                white_edge += 1
        col0 = 0
        for row0 in range(self.board_size):
            if board[row0][col0] == Piece.black:
                black_edge += 1
            elif board[row0][col0] == Piece.white:
                white_edge += 1
        col0 = self.board_size - 1
        for row0 in range(self.board_size):
            if board[row0][col0] == Piece.black:
                black_edge += 1
            elif board[row0][col0] == Piece.white:
                white_edge += 1
        return (black_edge - white_edge) / ((self.board_size - 2) * 4) * 100

    def get_danger_zone_score(self, board):
        black_zone = 0
        white_zone = 0
        row0 = 1
        for col0 in range(1, self.board_size - 1):
            if board[row0][col0] == Piece.black:
                black_zone += 1
            elif board[row0][col0] == Piece.white:
                white_zone += 1
        row0 = self.board_size - 2
        for col0 in range(1, self.board_size - 1):
            if board[row0][col0] == Piece.black:
                black_zone += 1
            elif board[row0][col0] == Piece.white:
                white_zone += 1
        col0 = 1
        for row0 in range(1, self.board_size - 1):
            if board[row0][col0] == Piece.black:
                black_zone += 1
            elif board[row0][col0] == Piece.white:
                white_zone += 1
        co10 = self.board_size - 2
        for row0 in range(1, self.board_size - 1):
            if board[row0][col0] == Piece.black:
                black_zone += 1
            elif board[row0][col0] == Piece.white:
                white_zone += 1
        return ((white_zone - black_zone) / 
                ((self.board_size - 2) * (self.board_size - 2) - 
                 (self.board_size - 4) * (self.board_size - 4))) * 100
    
    def get_center_4_score(self, board):
        black_count = 0
        white_count = 0
        if board[int(self.board_size / 2) - 1][int(self.board_size / 2) - 1] == Piece.black:
            black_count += 1
        elif board[int(self.board_size / 2) - 1][int(self.board_size / 2) - 1] == Piece.white:
            white_count += 1
        if board[int(self.board_size / 2) - 1][int(self.board_size / 2)] == Piece.black:
            black_count += 1
        elif board[int(self.board_size / 2) - 1][int(self.board_size / 2)] == Piece.white:
            white_count += 1
        if board[int(self.board_size / 2)][int(self.board_size / 2) - 1] == Piece.black:
            black_count += 1
        elif board[int(self.board_size / 2)][int(self.board_size / 2) - 1] == Piece.white:
            white_count += 1
        if board[int(self.board_size / 2)][int(self.board_size / 2)] == Piece.black:
            black_count += 1
        elif board[int(self.board_size / 2)][int(self.board_size / 2)] == Piece.white:
            white_count += 1
        return (black_count - white_count) / 4 * 100

    def get_line_score(self, board):
        black_lines = 0
        white_lines = 0
        for i in range(0, self.board_size):
            side = board[i][0]
            if side == Piece.black:
                black_lines += 1
            elif side == Piece.white:
                white_lines += 1
            else:
                continue
            for j in range(1, self.board_size):
                if board[i][j] != side:
                    if side == Piece.black:
                        black_lines -= 1
                    elif side == Piece.white:
                        white_lines -= 1
                    break
        for j in range(0, self.board_size):
            side = board[0][j]
            if side == Piece.black:
                black_lines += 1
            elif side == Piece.white:
                white_lines += 1
            else:
                continue
            for i in range(1, self.board_size):
                if board[i][j] != side:
                    if side == Piece.black:
                        black_lines -= 1
                    elif side == Piece.white:
                        white_lines -= 1
                    break
        return (black_lines - white_lines) / (self.board_size * 2) * 100

    def get_move_difference(self, board):
        black_moves = self.ai_get_legal_positions(board, "black")
        white_moves = self.ai_get_legal_positions(board, "white")
        if (len(black_moves) + len(white_moves)) <= 0:
            return 0
        else:
            return (len(white_moves) - len(black_moves)) / (len(black_moves) + len(white_moves)) * 100
    
    #returns a heurisitic evaluation of the board. 
    #Black is maximizing player, best score is 10000 (win)
    #White is minimizing player, best score is -10000 (win)
    def ai_heuristic(self, board, count_dict):
        score = 0
        #count_dict = self.ai_count_pieces(board)
        black_count = count_dict["black_count"]
        white_count = count_dict["white_count"]
        total_count = count_dict["total"]
        total_squares = self.board_size * self.board_size
        #check for winner
        if total_count == total_squares:
            if black_count > white_count:
                return 10000
            elif black_count < white_count:
                return -10000
            else:
                return 0
        if total_count < (self.board_size * self.board_size) * 1 / 2:
            count_score = (white_count - black_count) / total_squares * 100
        else:
            count_score = (black_count - white_count) / total_squares * 100
        corner_score = self.get_corner_score(board)
        x_square_score = self.get_x_squares_score(board)
        edge_score = self.get_edge_score(board)
        danger_zone_score = self.get_danger_zone_score(board)
        center_4_score = self.get_center_4_score(board)
        line_score = self.get_line_score(board)
        move_diff_score = self.get_move_difference(board)
        score = (count_score * 1 +
                 corner_score * 10 + 
                 x_square_score * 7 + 
                 edge_score * 3 + 
                 danger_zone_score * 1 + 
                 center_4_score * 2 + 
                 line_score * 2 +
                 move_diff_score * 2)
        return score


            


In [13]:
#complete random decision test
# game = Game(8)
# game.test_run_random()

#test edge
# game1 = Game(6)
# board1 = game1.setup_board(6)
# game1.ai_print_board(board1)
# board1 = game1.ai_add_piece(board1, "black", 2, 5)
# game1.ai_print_board(board1)
# board1 = game1.ai_add_piece(board1, "white", 1, 2)
# game1.ai_print_board(board1)
# board1 = game1.ai_add_piece(board1, "black", 1, 3)
# game1.ai_print_board(board1)
# board1 = game1.ai_add_piece(board1, "white", 1, 4)
# game1.ai_print_board(board1)

# game = Game(6)
# game.preface()
# game.ai_make_move()
# game.ai_make_move()
# game.ai_make_move()
# game.ai_make_move()
# game.ai_make_move()
# game.ai_make_move()
# game.ai_make_move()

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=4d302731-eb8d-41e8-80e5-f000b1dac4f5' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>